<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/LSTM%20CRF%20Model%20(ELMO%20Embeddings).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings Bi-LSTM-CRF Model

In [1]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro.py


In [2]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-qshecmez
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-qshecmez
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=d04e8f5d4f817df6c8b89739068d251d49e3e67b3a843a6275526a0171211af5
  Stored in directory: /tmp/pip-ephem-wheel-cache-zdjcumm8/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [3]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 757kB 5.0MB/s 


In [4]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

Using TensorFlow backend.


In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [6]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words= [w.lower() for w in words]

n_words = len(set(words))
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  24277
Number of Labels:  5


In [7]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["ORG"]))

The labels LOC (location) is identified by the index: 5


In [8]:
word2idx = {w: i + 2 for i, w in enumerate(list(set(words)))}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 3954


In [0]:
def replace_number(word):
  if any(i.isdigit() for i in word):
    return "d"
  else:
    return word

In [10]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [0]:
sentences = [[w[0] for w in s] for s in train]

new_X = []
for seq in sentences:
    new_seq = []
    for i in range(MAX_LEN):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
X = new_X

batch_size = 200
div = int(np.floor(len(X)/batch_size))

X = X[:div*batch_size]

In [12]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([5, 4, 2, 4, 4, 4, 2, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]
y = y[:div*batch_size]

## ELMO Embeddings

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
from keras.layers.merge import add
from keras.layers import Lambda

In [0]:
sess = tf.Session()
K.set_session(sess)

In [0]:
elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=False)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [0]:
max_len = MAX_LEN
def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.cast(tf.count_nonzero(x, axis=1), dtype=tf.int32)
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]

In [18]:
# Model definition

input = Input(shape=(MAX_LEN,),dtype=tf.string)

embedding = Lambda(ElmoEmbedding, output_shape=(None,None,MAX_LEN,1024))(input)

lstm1 = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(embedding)

lstm2 = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(lstm1)

model_2 = TimeDistributed(Dense(10, activation="relu"))(lstm2)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model_2)  # output
model_2 = Model(input, out)

W0826 11:26:12.344289 139913001281408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 11:26:12.352726 139913001281408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 11:26:12.362220 139913001281408 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/deprecation.py:507: calling count_nonzero (from tensorflow.python.ops.math_ops) with axis is deprecated and will be removed in a future version.
Instructions for updating:
reduction_indices is deprecated, use axis instead
W0826 11:26:12.822434 139913001281408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random

In [19]:
model_2.compile(optimizer='adam', loss=crf_loss,metrics=[crf_viterbi_accuracy])
model_2.summary()

W0826 11:26:16.611124 139913001281408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 11:26:16.717531 139913001281408 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:2403: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 124)               0         
_________________________________________________________________
lambda_1 (Lambda)            (None, None, None, 124, 1 0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         557568    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 10)          1290      
_________________________________________________________________
crf_1 (CRF)                  (None, None, 6)           114       
Total params: 657,788
Trainable params: 657,788
Non-trainable params: 0
_________________________________________________________________


In [20]:
%%time
BATCH_SIZE = 200
EPOCHS=10

history = model_2.fit(np.array(X), np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

W0826 11:26:23.444662 139913001281408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 11200 samples, validate on 2800 samples
Epoch 1/10
11200/11200 [==============================] - 297s 27ms/step - loss: 0.5271 - crf_viterbi_accuracy: 0.8069 - val_loss: 0.0870 - val_crf_viterbi_accuracy: 0.9809
Epoch 2/10
11200/11200 [==============================] - 289s 26ms/step - loss: 0.0510 - crf_viterbi_accuracy: 0.9861 - val_loss: 0.0332 - val_crf_viterbi_accuracy: 0.9903
Epoch 3/10
11200/11200 [==============================] - 288s 26ms/step - loss: 0.0225 - crf_viterbi_accuracy: 0.9938 - val_loss: 0.0194 - val_crf_viterbi_accuracy: 0.9938
Epoch 4/10
11200/11200 [==============================] - 289s 26ms/step - loss: 0.0134 - crf_viterbi_accuracy: 0.9958 - val_loss: 0.0134 - val_crf_viterbi_accuracy: 0.9953
Epoch 5/10
11200/11200 [==============================] - 288s 26ms/step - loss: 0.0088 - crf_viterbi_accuracy: 0.9970 - val_loss: 0.0102 - val_crf_viterbi_accuracy: 0.9966
Epoch 6/10
11200/11200 [==============================] - 288s 26ms/step - loss: 0.006

## Predict on Training Set

In [21]:
pred = model_2.predict(np.array(X),verbose =1)

14000/14000 [==============================] - 283s 20ms/step


In [22]:
# TRain Eval
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

              precision    recall  f1-score   support

         LOC       0.97      0.96      0.96      8261
        MISC       0.94      0.90      0.92      4587
           O       1.00      1.00      1.00    169339
         ORG       0.93      0.96      0.94     10011
         PAD       1.00      1.00      1.00   1532754
         PER       0.98      0.99      0.98     11048

    accuracy                           1.00   1736000
   macro avg       0.97      0.97      0.97   1736000
weighted avg       1.00      1.00      1.00   1736000

0.9989247526249677


In [23]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

         LOC       0.97      0.96      0.96      8261
        MISC       0.94      0.90      0.92      4587
         PER       0.98      0.99      0.98     11048
         ORG       0.93      0.96      0.94     10011

   micro avg       0.96      0.96      0.96     33907
   macro avg       0.96      0.95      0.95     33907
weighted avg       0.96      0.96      0.96     33907

0.9584404269612038


## Predict on Test Set

In [0]:
sentences = [[w[0] for w in s] for s in test]

new_X = []
for seq in sentences:
    new_seq = []
    for i in range(MAX_LEN):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
X_test = new_X

# Convert Tag/Label to tag_index
y_test = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]

# Padding each sentence to have the same lenght
y_test = pad_sequences(maxlen=MAX_LEN, sequences=y_test, padding="post", value=tag2idx["PAD"])

# One-Hot encode categorical labels
y_test = [to_categorical(i, num_classes=n_tags+1) for i in y_test]

In [25]:
pred = model_2.predict(np.array(X_test), verbose=1)

3453/3453 [==============================] - 70s 20ms/step


In [26]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y_test, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)


              precision    recall  f1-score   support

         LOC       0.92      0.89      0.90      1925
        MISC       0.75      0.76      0.75       918
         PER       0.97      0.97      0.97      2773
         ORG       0.84      0.90      0.87      2496

   micro avg       0.89      0.91      0.90      8112
   macro avg       0.87      0.88      0.87      8112
weighted avg       0.89      0.91      0.90      8112

0.8981677540092333
